In [1]:
import math
import random
import numpy as np

def get_attacks(objects_in_line):
    
    tot_at = 0
    for elem in range(objects_in_line):
        tot_at += elem
    
    return tot_at



class n_queen_solver:
    def __init__(self, board_size, n_pair_of_parents, n_children, mutation_factor = 100):
        self.board_size = board_size
        self.max_attacks = get_attacks(self.board_size)
        
        assert n_pair_of_parents >= 1, "NUMBER OF PAIR OF PARENTS MUST BE GREATER OR EQUAL THAN 1"
        self.n_2_parents = n_pair_of_parents
        assert n_children >= 1, "NUMBER OF CHILDREN PER PAIR OF PARENTS MUST BE GREATER OR EQUAL THAN 1"
        self.n_children = n_children
        
        self.living_boards = self._initiate_parents()
        
        self.scores = []
        for board in self.living_boards:
            self.scores.append(self._get_score(board))
        
        self.mutation_factor = mutation_factor
        
        return
    
    
    
    def _initiate_parents(self):
        
        n_cases = self.n_2_parents*self.n_children
        living_boards = []
        
        for _ in range(n_cases):
            board = []
            for __ in range(self.board_size):
                board.append(random.choice(range(self.board_size)))
            
            living_boards.append(board)
        
        return living_boards
    
    
    
    def _get_score(self, board):
        return self.max_attacks - self._get_attacks(board)
    
    
    
    def _get_attacks(self, board):
        
        total_attacks = 0
        
        #We check HORIZONTAL attacks
        horizontal_dict = {}
        for i in range(self.board_size):
            aux_val = board[i]
            if aux_val in horizontal_dict.keys():
                horizontal_dict[aux_val] += 1
            else:
                horizontal_dict[aux_val] = 1
        
        for val in horizontal_dict.keys():
            if horizontal_dict[val] > 1:
                total_attacks += get_attacks(horizontal_dict[val])
        
        #We check DIAGONAL attacks                      [1 2 0 4 3]
        for i in range(self.board_size):
            for mode in range(2):
                aux_attacks = 0
                for j in range(1,self.board_size-i):
                    if mode == 0:
                        pos_i = i
                        pos_j = i+j
                        val_j = board[i]+j
                    elif mode == 1:
                        pos_i = i
                        pos_j = i+j
                        val_j = board[i]-j
                    
                    if val_j < 0 or val_j > self.board_size - 1:
                        break
                    
                    if board[pos_j] == val_j:
                        aux_attacks += 1
                
                if aux_attacks > 0:
                    total_attacks += aux_attacks
        
        return total_attacks
    
    
    
    def select_parents(self):
        
        accumulative_weights = []
        tot_acc = 0
        for score in self.scores:
            tot_acc += score
            accumulative_weights.append(tot_acc)
        
        list_parents = []
        for _ in range(self.n_2_parents):
            
            pair_parents = []
            for __ in range(2):
                pos = random.choice(range(tot_acc))
                
                for i,acc_weight in enumerate(accumulative_weights):
                    
                    if pos < acc_weight:
                        pair_parents.append(self.living_boards[i])
                        break
            
            list_parents.append(pair_parents)
        
        return list_parents
    
    
    
    def create_children(self, parent_list):
        
        n_board = self.board_size
        
        pairs_of_children = (self.n_children+1) // 2
        extra_child = self.n_children % 2
        
        found_best = [False, None, 0]
        
        list_of_children = []
        list_of_scores = []
        n_children = 0
        for pair in parent_list:
            for n_child in range(pairs_of_children):
                aux_cut = random.choice(range(1,n_board))
                
                aux_part1 = pair[0][0:aux_cut]
                aux_part2 = pair[1][aux_cut:n_board]
                aux_1 = np.concatenate((aux_part1, aux_part2), axis=0)
                aux_1 = self.mutation_step(aux_1)
                score_1 = self._get_score(aux_1)
                
                aux_part1 = pair[1][0:aux_cut]
                aux_part2 = pair[0][aux_cut:n_board]
                aux_2 = np.concatenate((aux_part1, aux_part2), axis=0)
                aux_2 = self.mutation_step(aux_2)
                score_2 = self._get_score(aux_2)
            
            
            if self.n_children - n_children:     #IF THERE IS ONLY 1 CHILD LEFT
                list_of_children.append(aux_1)
                list_of_scores.append(score_1)
                if score_1 == self.max_attacks:
                    found_best = [True, aux_1, score_1]
                
                n_children += 1
            else:                                #IF AT LEAST A PAIR OF CHILDREN IS LEFT
                list_of_children.append(aux_1)
                list_of_scores.append(score_1)
                if score_1 == self.max_attacks:
                    found_best = [True, aux_1, score_1]
                
                list_of_children.append(aux_2)
                list_of_scores.append(score_2)
                if score_2 == self.max_attacks:
                    found_best = [True, aux_2, score_2]
                
                n_children += 2
        
        self.living_boards = list_of_children
        self.scores = list_of_scores
        
        return found_best
    
    
    
    def mutation_step(self, board):
        
        if self.mutation_factor == 0 or self.mutation_factor == None: # --> NO MUTATIONS
            return board
        
        val = random.choice(range(self.mutation_factor))
        if val == 0:
            pos = random.choice(range(self.board_size))
            board[pos] = random.choice(range(self.board_size))
        
        return board
    
    
    
    def execute_one_generation(self):
        
        parents = self.select_parents()
        found, best_board, best_score = self.create_children(parents)
        
        return found, best_board, best_score
    
    
    
    def find_best(self):
        
        i = random.choice(range(len(self.living_boards)))
        best_board = self.living_boards[i]
        best_score = self.scores[i]
        
        for i,board in enumerate(self.living_boards):
            if self.scores[i] > best_score:
                best_board = board
                best_score = self.scores[i]
        
        return best_board, best_score
    
    
    
    def execute_algorithm(self, n_generations = 100):
        
        for i in range(n_generations):
            found, best_board, best_score = self.execute_one_generation()

            if found:
                return best_board, best_score
        
        best_board, best_score = self.find_best()
        
        return best_board, best_score
    
    

if __name__ == "__main__":
    
    solver = n_queen_solver(board_size = 6, n_pair_of_parents = 2, n_children = 4, mutation_factor = 10)
    
    print("\nSTARTING BOARDS:",solver.living_boards)
    print("INITIAL SCORES FOR PREVIOUS BOARDS:",solver.scores)
    
    best_board, best_score = solver.execute_algorithm(10000)
    print("\nBEST SOLUTION FOUND:",best_board,"| Score =",best_score,"| Best Score =",get_attacks(6))
    
    


STARTING BOARDS: [[2, 4, 0, 2, 0, 3], [1, 0, 5, 0, 1, 2], [2, 3, 1, 3, 2, 3], [4, 2, 1, 5, 1, 0], [1, 5, 4, 5, 2, 1], [0, 5, 0, 1, 3, 5], [0, 4, 5, 1, 3, 4], [4, 5, 4, 4, 0, 3]]
INITIAL SCORES FOR PREVIOUS BOARDS: [10, 8, 8, 12, 6, 11, 11, 9]

BEST SOLUTION FOUND: [2 5 1 4 0 3] | Score = 15 | Best Score = 15
